# Applying Classification Modeling
The goal of this week's assessment is to find the model which best predicts whether or not a person will default on their bank loan. In doing so, we want to utilize all of the different tools we have learned over the course: data cleaning, EDA, feature engineering/transformation, feature selection, hyperparameter tuning, and model evaluation. 


#### Data Set Information:

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel Sorting Smoothing Method to estimate the real probability of default. With the real probability of default as the response variable (Y), and the predictive probability of default as the independent variable (X), the simple linear regression result (Y = A + BX) shows that the forecasting model produced by artificial neural network has the highest coefficient of determination; its regression intercept (A) is close to zero, and regression coefficient (B) to one. Therefore, among the six data mining techniques, artificial neural network is the only one that can accurately estimate the real probability of default. 

- NT is the abbreviation for New Taiwain. 


#### Attribute Information:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 
    - X6 = the repayment status in September, 2005; 
    - X7 = the repayment status in August, 2005; . . .;
    - etc...
    - X11 = the repayment status in April, 2005. 
    - The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). 
    - X12 = amount of bill statement in September, 2005;
    - etc...
    - X13 = amount of bill statement in August, 2005; . . .; 
    - X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). 
    - X18 = amount paid in September, 2005; 
    - X19 = amount paid in August, 2005; . . .;
    - etc...
    - X23 = amount paid in April, 2005. 




You will fit three different models (KNN, Logistic Regression, and Decision Tree Classifier) to predict credit card defaults and use gridsearch to find the best hyperparameters for those models. Then you will compare the performance of those three models on a test set to find the best one.  


## Process/Expectations

- You will be working in pairs for this assessment

### Please have ONE notebook and be prepared to explain how you worked in your pair.

1. Clean up your data set so that you can perform an EDA. 
    - This includes handling null values, categorical variables, removing unimportant columns, and removing outliers.
2. Perform EDA to identify opportunities to create new features.
    - [Great Example of EDA for classification](https://www.kaggle.com/stephaniestallworth/titanic-eda-classification-end-to-end) 
    - [Using Pairplots with Classification](https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166)
3. Engineer new features. 
    - Create polynomial and/or interaction features. 
    - Additionaly, you must also create **at least 2 new features** that are not interactions or polynomial transformations. 
        - *For example, you can create a new dummy variable that based on the value of a continuous variable (billamount6 >2000) or take the average of some past amounts.*
4. Perform some feature selection. 
    
5. You must fit **three** models to your data and tune **at least 1 hyperparameter** per model. 
6. Using the F-1 Score, evaluate how well your models perform and identify your best model.
7. Using information from your EDA process and your model(s) output provide insight as to which borrowers are more likely to deafult


In [37]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


## 1. Data Cleaning

In [38]:
'''
X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.

X2: Gender (1 = male; 2 = female).

X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).

X4: Marital status (1 = married; 2 = single; 3 = others).

X5: Age (year).

X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.

X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005.

X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005.

Y: default on payments : 1- yes. 0- no

more on data dictionary at: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
''';

In [39]:
df = pd.read_csv('training_data.csv',index_col = 0)

In [40]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
28835,220000,2,1,2,36,0,0,0,0,0,...,221193,181859,184605,10000,8018,10121,6006,10987,143779,1
25329,200000,2,3,2,29,-1,-1,-1,-1,-1,...,326,326,326,326,326,326,326,326,326,0
18894,180000,2,1,2,27,-2,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
690,80000,1,2,2,32,0,0,0,0,0,...,43882,42256,42527,1853,1700,1522,1548,1488,1500,0
6239,10000,1,2,2,27,0,0,0,0,0,...,5444,2639,2697,2000,1100,600,300,300,1000,1


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22500 entries, 28835 to 7624
Data columns (total 24 columns):
X1     22500 non-null object
X2     22500 non-null object
X3     22500 non-null object
X4     22500 non-null object
X5     22500 non-null object
X6     22500 non-null object
X7     22500 non-null object
X8     22500 non-null object
X9     22500 non-null object
X10    22500 non-null object
X11    22500 non-null object
X12    22500 non-null object
X13    22500 non-null object
X14    22500 non-null object
X15    22500 non-null object
X16    22500 non-null object
X17    22500 non-null object
X18    22500 non-null object
X19    22500 non-null object
X20    22500 non-null object
X21    22500 non-null object
X22    22500 non-null object
X23    22500 non-null object
Y      22500 non-null object
dtypes: object(24)
memory usage: 4.3+ MB


In [45]:
cols = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23','Y']

df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [46]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
28835,220000.0,2.0,1.0,2.0,36.0,0.0,0.0,0.0,0.0,0.0,...,221193.0,181859.0,184605.0,10000.0,8018.0,10121.0,6006.0,10987.0,143779.0,1.0
25329,200000.0,2.0,3.0,2.0,29.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,0.0
18894,180000.0,2.0,1.0,2.0,27.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
690,80000.0,1.0,2.0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,...,43882.0,42256.0,42527.0,1853.0,1700.0,1522.0,1548.0,1488.0,1500.0,0.0
6239,10000.0,1.0,2.0,2.0,27.0,0.0,0.0,0.0,0.0,0.0,...,5444.0,2639.0,2697.0,2000.0,1100.0,600.0,300.0,300.0,1000.0,1.0


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22500 entries, 28835 to 7624
Data columns (total 24 columns):
X1     22499 non-null float64
X2     22499 non-null float64
X3     22499 non-null float64
X4     22499 non-null float64
X5     22499 non-null float64
X6     22499 non-null float64
X7     22499 non-null float64
X8     22499 non-null float64
X9     22499 non-null float64
X10    22499 non-null float64
X11    22499 non-null float64
X12    22499 non-null float64
X13    22499 non-null float64
X14    22499 non-null float64
X15    22499 non-null float64
X16    22499 non-null float64
X17    22499 non-null float64
X18    22499 non-null float64
X19    22499 non-null float64
X20    22499 non-null float64
X21    22499 non-null float64
X22    22499 non-null float64
X23    22499 non-null float64
Y      22500 non-null object
dtypes: float64(23), object(1)
memory usage: 4.3+ MB


In [32]:
df.head()

,given_credit,gender,education,mar_status,age,paystatus_09,paystatus_08,paystatus_07,paystatus_06,paystatus_05,...,billamt_06,billamt_05,billamt_04,payamt_09,payamt_08,payamt_07,payamt_06,payamt_05,payamt_04,defaulted
28835,220000,2,1,2,36.0,0,0,0,0,0,...,221193.0,181859.0,184605.0,10000.0,8018.0,10121.0,6006.0,10987.0,143779.0,1
25329,200000,2,3,2,29.0,-1,-1,-1,-1,-1,...,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,326.0,0
18894,180000,2,1,2,27.0,-2,-2,-2,-2,-2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
690,80000,1,2,2,32.0,0,0,0,0,0,...,43882.0,42256.0,42527.0,1853.0,1700.0,1522.0,1548.0,1488.0,1500.0,0
6239,10000,1,2,2,27.0,0,0,0,0,0,...,5444.0,2639.0,2697.0,2000.0,1100.0,600.0,300.0,300.0,1000.0,1


In [36]:
df.paystatus_04.isnull().sum()

0

In [34]:
df[['given_credit',
    'gender','education',
    'mar_status',
    'age',
    'paystatus_09',
    'paystatus_08',
    'paystatus_07',
    'paystatus_06',
    'paystatus_05',
    'paystatus_04',
    'defaulted']].astype('int')

ValueError: invalid literal for int() with base 10: 'LIMIT_BAL'

In [13]:
df.Y.value_counts() # class imbalance in default.

0                             17471
1                              5028
default payment next month        1
Name: Y, dtype: int64

In [15]:
rnm_dict = {'X1' :'given_credit',
            'X2' :'gender',
            'X3' :'education',
            'X4' :'mar_status',
            'X5' :'age',
            'X6' :'paystatus_09',
            'X7' :'paystatus_08',
            'X8' :'paystatus_07',
            'X9' :'paystatus_06',
            'X10':'paystatus_05',
            'X11':'paystatus_04',
            'X12':'billamt_09',
            'X13':'billamt_08',
            'X14':'billamt_07',
            'X15':'billamt_06',
            'X16':'billamt_05',
            'X17':'billamt_04',
            'X18':'payamt_09', 
            'X19':'payamt_08',
            'X20':'payamt_07',
            'X21':'payamt_06',
            'X22':'payamt_05',
            'X23':'payamt_04',
            'Y':'defaulted'}



In [16]:
df.rename(columns = rnm_dict, inplace = True)

In [18]:
df.columns

Index(['given_credit', 'gender', 'education', 'mar_status', 'age',
       'paystatus_09', 'paystatus_08', 'paystatus_07', 'paystatus_06',
       'paystatus_05', 'paystatus_04', 'billamt_09', 'billamt_08',
       'billamt_07', 'billamt_06', 'billamt_05', 'billamt_04', 'payamt_09',
       'payamt_08', 'payamt_07', 'payamt_06', 'payamt_05', 'payamt_04',
       'defaulted'],
      dtype='object')

In [17]:
df.gender.value_counts() # class imbalance, appropriate choices are represented.

2      13572
1       8927
SEX        1
Name: gender, dtype: int64

In [22]:
df.education.value_counts()

2            10516
1             7919
3             3713
5              208
4               90
6               42
0               11
EDUCATION        1
Name: education, dtype: int64

In [21]:
# no information for extraneous values could be found. group all education values of 0,5,6
# as 4's since 4 represents 'other' for education extent.
four_dict = {0:4, 5:4, 6:4}
df.education = df.education.replace(four_dict)

In [26]:
#verify change 
df.education.value_counts()

2            10516
1             7919
3             3713
5              208
4               90
6               42
0               11
EDUCATION        1
Name: education, dtype: int64

In [11]:
# Split data to be used in the models
# Create matrix of features
X = df.drop('defaulted', axis = 1) # grabs everything else but 'Survived'


# Create target variable
y = df['defaulted'] # y is the column we're trying to predict

## 2. EDA

## 3. Feature Engineering

## 4. Feature Selection

## 5. Model Fitting and Hyperparameter Tuning
KNN, Logistic Regression, Decision Tree

## 6. Model Evaluation

## 7. Final Model